In [ ]:
"""
Goal:
Visualize cost as requests per month go up and as duration per request
changes for a fixed memory size

Parameters: 
* number of requests / month
* average duration of execution
* memory

$2*10^-7 per request
1M requests free
1/6 * 10^-4 ($0.0000166667) per GB-s
2^2 * 10^5 GB-s free
2^-3 GB (128MB) * 10^-1s minimum invocation
"""

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, Dropdown

%matplotlib notebook

REQUEST_COST = 0.0000002  # 2*10^-7
FREE_REQUESTS = 1_000_000  # 10^7
GB_S_COST = 0.0000166667  # (1/6) * 10^-4
MB_PER_GB = 2^10
FREE_GB_S = 400_000
MIN_DURATION_SECS = 0.1
DURATION_STEP_SECS = 0.1
MAX_DURATION_SECS = 1
MIN_MEM_MB = 128
MAX_MEM_MB = 3008
MEM_STEP_MB = 64

In [9]:
def plot_costs(max_request_count, memory_mb):
    fig = plt.figure()
    
    ax = fig.add_subplot(111, projection='3d')
    # plot_total_cost(ax, max_request_count, memory_mb)
    plot_request_cost(ax, max_request_count, memory_mb)
    plot_compute_cost(ax, max_request_count, memory_mb)
    
    plt.show()
    
def plot_total_cost(ax, max_request_count, memory_mb):
    x, y = make_x_y(max_request_count)
    z = v_calc_total_cost(x, y, memory_mb)
    
    cmap = cm.coolwarm
    cmap.set_under(color='green') 
    surf = ax.plot_surface(x, y, z, cmap=cmap, linewidth=0, antialiased=False, vmin=0.0000001)
    
def plot_request_cost(ax, max_request_count, memory_mb):
    x, y = make_x_y(max_request_count)
    z = v_calc_request_cost(x, y, memory_mb)
    
    cmap = cm.coolwarm
    cmap.set_under(color='green') 
    surf = ax.plot_surface(x, y, z, cmap=cmap, linewidth=0, antialiased=False, vmin=0.0000001)
    
def plot_compute_cost(ax, max_request_count, memory_mb):
    x, y = make_x_y(max_request_count)
    z = v_calc_compute_cost(x, y, memory_mb)
    
    cmap = cm.coolwarm
    cmap.set_under(color='green') 
    surf = ax.plot_surface(x, y, z, cmap=cmap, linewidth=0, antialiased=False, vmin=0.0000001)
    
def make_x_y(max_request_count):
    x = np.linspace(0, max_request_count, 1000)
    y = np.arange(MIN_DURATION_SECS, MAX_DURATION_SECS, DURATION_STEP_SECS)
    return np.meshgrid(x, y)

def calc_total_cost(request_count, average_duration_secs, memory_mb):
    return (
        calc_request_cost(request_count) + 
        calc_compute_cost(request_count, average_duration_secs, memory_mb)
    )
    
def calc_request_cost(request_count, average_duration_secs=0, memory_mb=0):
    cost_request_count = max(request_count - FREE_REQUESTS, 0)
    return cost_request_count * REQUEST_COST

def calc_compute_cost(request_count, average_duration_secs, memory_mb):
    memory_gb = memory_mb / MB_PER_GB
    compute_time_secs = request_count * average_duration_secs
    compute_gb_s = memory_gb * compute_time_secs
    cost_compute_gb_s = max(compute_gb_s - FREE_GB_S, 0)
    return cost_compute_gb_s * GB_S_COST

v_calc_total_cost = np.vectorize(calc_total_cost)
v_calc_request_cost = np.vectorize(calc_request_cost)
v_calc_compute_cost = np.vectorize(calc_compute_cost)

In [10]:
plt.close("all")
interact(
    plot_costs, 
    max_request_count=IntSlider(min=100_000, max=100_000_000, step=1000, value=2_000_000), 
    memory_mb=Dropdown(
        options=range(MIN_MEM_MB, MAX_MEM_MB + MEM_STEP_MB, MEM_STEP_MB),
        value=MIN_MEM_MB,
    ),
) 

interactive(children=(IntSlider(value=2000000, description='max_request_count', max=100000000, min=100000, ste…

<function __main__.plot_costs(max_request_count, memory_mb)>